In [112]:
from openai import OpenAI
import gradio as gr
import json
from dotenv import load_dotenv
import os

In [113]:
load_dotenv(override=True)
openai = OpenAI()

In [114]:
system_prompt = """
You are helpful assistant for an airline company. Give short, courteous answers, no more than 1 sentence. 
Always be accurate. If you don't know the answer, strictly mention the same and DO NOT MAKE INFORMATION UP.
"""

In [115]:
def chat(message, history):
    messages = []
    messages.append({"role": "system", "content": system_prompt})
    for user_prompt, assistant_response in history:
        messages.append({"role": "user", "content": user_prompt})
        messages.append({"role": "assistant", "content": assistant_response})
    messages.append({"role": "user", "content": message})

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return response.choices[0].message.content

In [117]:
gr.ChatInterface(
    fn=chat,
    title="Airline Assistant",
    description="Ask me anything about our airline services.",
).launch()

c:\TFS\Study\machine-learning\.venv\lib\site-packages\gradio\chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


In [118]:
ticket_prices = {"london": "$299", "paris": "$199", "new york": "$399", "tokyo": "$499", "delhi": "$299"}
def get_ticket_prices(destination):
    destination = destination.lower()
    price = ticket_prices.get(destination, "Unknown")
    return price

In [119]:
ticket_price_tool = {
    "name": "get_ticket_prices",
    "description": "Get ticket price for a given destination. Call this whenever you need to know the ticket price. For example, 'What is the ticket price for London?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination": {
                "type": "string",
                "description": "The destination for which you want to know the ticket price.",
            },
        },
        "required": ["destination"],
        "additionalProperties": False
    }
}

In [120]:
tools = [{"type": "function", "function": ticket_price_tool}]

In [121]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    tool_args = json.loads(tool_call.function.arguments)

    if tool_call.function.name == "get_ticket_prices":
        destination = tool_args["destination"]
        price = get_ticket_prices(destination)
        response = {
            "role": "tool",
            "content": json.dumps({"destination": destination, "price": price}),
            "tool_call_id": tool_call.id
        }
        return response

In [125]:
def chat_with_tools(message, history):
    messages = []
    messages.append({"role": "system", "content": system_prompt})
    for user_prompt, assistant_response in history:
        messages.append({"role": "user", "content": user_prompt})
        messages.append({"role": "assistant", "content": assistant_response})
    messages.append({"role": "user", "content": message})

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools
    )

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )

    return response.choices[0].message.content

In [126]:
gr.ChatInterface(
    fn=chat_with_tools,
    title="Airline Assistant",
    description="Ask me anything about our airline services.",
).launch(share=True)

c:\TFS\Study\machine-learning\.venv\lib\site-packages\gradio\chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7871

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
